### 1. Flume概述

- 日志采集和汇总工具
- 收集到的日志数据汇总到HDFS存储

### 2. 安装文件

- 云盘
- 官方下载
- flume：1.9.0


### 3.  Flume组件

- source：数据源（需要采集的数据）
- channel：临时存储的数据位置，通常存储在内存中
- sink：数据目标存储，hdfs



### 4. 安装Flume

- 上传安装文件到服务器
- 解压安装，命令如下：

In [ ]:
    $ tar zxvf apache-flume-1.9.0-bin.tar.gz
    
    $ ll
    
    drwxrwxr-x. 7 hadoop hadoop       187 7月   7 17:22 apache-flume-1.9.0-bin
    -rwxrwxr-x. 1 hadoop hadoop  67938106 7月   7 17:21 apache-flume-1.9.0-bin.tar.gz
    -rwxrwxr-x. 1 hadoop hadoop 359196911 7月   7 11:19 hadoop-3.2.1.tar.gz
    -rw-r--r--. 1 hadoop hadoop 194151339 7月   7 14:46 jdk-8u231-linux-x64.tar.gz
        
    $ exit
    
    # mv apache-flume-1.9.0-bin /usr/
    
    

- 配置环境变量，~/.bash_profile, 具体内容：


In [ ]:
    JAVA_HOME=/usr/jdk1.8.0_231
    HADOOP_HOME=/usr/hadoop-3.2.1
    FLUME_HOME=/usr/apache-flume-1.9.0-bin
    PATH=$FLUME_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin:$JAVA_HOME/bin:$PATH

    export JAVA_HOME
    export HADOOP_HOME
    export FLUME_HOME
    export PATH

    注意：source ~/.bash_profile

- Flume基本配置，$FLUME_HOME/conf/flume-env.sh, 具体配置如下：

In [ ]:
    $ cp flume-env.sh.template  flume-env.sh
    $ vi flume-env.sh
    
    
     22 export JAVA_HOME=/usr/jdk1.8.0_231

- 解决jar包冲突问题


In [ ]:
    $ cd /usr/apache-flume-1.9.0-bin/lib
    $ ll guava-11.0.2.jar 
    -rw-rw-r--. 1 hadoop hadoop 1648200 9月  13 2018 guava-11.0.2.jar
    
    
    $ cd /usr/hadoop-3.2.1/share/hadoop/common/lib
    $ ll guava-27.0-jre.jar 
    -rw-r--r--. 1 hadoop hadoop 2747878 9月  10 2019 guava-27.0-jre.jar
    
    
    $ rm -rf /usr/apache-flume-1.9.0-bin/lib/guava-11.0.2.jar 
    $ cp /usr/hadoop-3.2.1/share/hadoop/common/lib/guava-27.0-jre.jar /usr/apache-flume-1.9.0-bin/lib

### 5. 实现数据同步

- 功能需求

    1. 采集爬虫服务器中的数据汇总到hdfs指定目录
    
- 实现步骤
    1. 启动数据采集服务
    2. 启动hdfs服务，保证hdfs可读写
    3. 配置一个Agent（Source、Channel、Sink）
    4. 使用flume1.7+版本新特性，source组件中提供了高可靠的同步模式(TIALDIR), 保证数据不丢失
    5. 编写运行脚本(shell/sh)并执行
    
    

In [ ]:
    $ cd /usr/apache-flume-1.9.0-bin/
    $ mkdir myconf
    $ cd myconf
    
    $ vi flume-taildir-memory-hdfs.properties
    
    

    创建agent配置文件"flume-taildir-memory-hdfs.properties", 具体配置如下：

In [ ]:

# Name the components on this agent
hdfs_agent.sources = r1
hdfs_agent.sinks = k1
hdfs_agent.channels = c1

# Describe/configure the source
hdfs_agent.sources.r1.type = TAILDIR
hdfs_agent.sources.r1.filegroups = f1
hdfs_agent.sources.r1.filegroups.f1 = /home/hadoop/spider/data/collect/.*\.log
hdfs_agent.sources.r1.positionFile = /home/hadoop/spider/data/.flume/taildir_position.json

# Describe the sink
hdfs_agent.sinks.k1.type = hdfs
hdfs_agent.sinks.k1.hdfs.path = hdfs://hadoop:9000/flume/hdfs_filegroups_source/%Y-%m-%d/
hdfs_agent.sinks.k1.hdfs.rollInterval = 3600
hdfs_agent.sinks.k1.hdfs.rollSize = 1048576
hdfs_agent.sinks.k1.hdfs.rollCount = 0
hdfs_agent.sinks.k1.hdfs.filePrefix = log_file_%H
hdfs_agent.sinks.k1.hdfs.fileSuffix = .log
hdfs_agent.sinks.k1.hdfs.fileType = DataStream
hdfs_agent.sinks.k1.hdfs.useLocalTimeStamp = true

# Use a channel which buffers events in memory
hdfs_agent.channels.c1.type = memory
hdfs_agent.channels.c1.capacity = 1000
hdfs_agent.channels.c1.transactionCapacity = 100

# Bind the source and sink to the channel
hdfs_agent.sources.r1.channels = c1
hdfs_agent.sinks.k1.channel = c1

In [ ]:
    $ hdfs dfs -mkdir -p /flume/hdfs_filegroups_source/

    编写shell脚本，安装目录下创建mysbin目录，start_taildir_memory_hdfs.sh ， 具体内容如下：

In [ ]:
    $ cd /usr/apache-flume-1.9.0-bin/
    $ mkdir mysbin
    $ cd mysbin
    $ vi start_taildir_memory_hdfs.sh
    
    
    #!/bin/bash

    ROOT_PATH=$(dirname $(dirname $(readlink -f $0)))
    cd $ROOT_PATH

    bin/flume-ng agent --conf ./conf/ -f myconf/flume-taildir-memory-hdfs.properties -Dflume.root.logger=INFO,console -n hdfs_agent

    更改脚本的执行权限，命令如下：
    

In [ ]:
    $ chmod 755 start_taildir_memory_hdfs.sh
     ll
    总用量 4
    -rwxr-xr-x. 1 hadoop hadoop 206 7月   7 17:56 start_taildir_memory_hdfs.sh

    执行start_taildir_memory_hdfs.sh脚本文件，命令如下：
    

In [ ]:
    $ nohup ./start_taildir_memory_hdfs.sh &
    